In [1]:
import numpy as np
import pandas as pd
import os

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# 관련 라이브러리 임포트 
import matplotlib.font_manager as fm

#  한글글꼴로 변경
# plt.rcParams['font.family'] = '한글글꼴명'
plt.rcParams['font.size'] = 11.0
# plt.rcParams['font.family'] = 'batang'
plt.rcParams['font.family'] = 'Malgun Gothic'

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
matplotlib.rcParams['axes.unicode_minus'] = False
plt.style.use('ggplot')

# 그래프 기본 크기 설정 
plt.rcParams['figure.figsize'] = [10, 10]

In [3]:
from sklearn.ensemble import VotingClassifier, VotingRegressor

from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import f1_score, recall_score, precision_score, classification_report, r2_score

from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.pipeline import Pipeline

# 앙상블(Ensemble) 

- Ensemble : 조화라는 사전적인 의미를 지님
- Ensemble Learning
    - 여러 개의 기본 모델을 활용하여 하나의 새로운 모델을 만들어내는 개념 
    - 기본모델(Base Model)을 weak learner, classifier, base 

###  보팅(Voting) 
    - 여러개의 분류기가 투표를 통해 최종 예측 결과를 결정하는 방식 
    - 하드보팅 : 다수의 classifier 간 다수결의 최종 class 결정 
    - 소프트보팅 : 다수의 classfier 들의 class 확률을 평균하여 결정 
    - Voting Classifier¶ 활용 

    모델=[(키1,예측기1),(키2,예측기2)] , voting='soft/hard' )


# Voting Classification => 분류



<img src='https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fb5UNZc%2FbtrhVZEJkJA%2F1bqKVdovvqY2NY050MWMSK%2Fimg.jpg'>

**위스콘신 유방암 데이터 로드**

- cancer.data
- cancer.feature_names
- cancer.target 

In [4]:
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer()
cancer.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [6]:
# X 설정
X = cancer.data

In [7]:
# y 설정
y = cancer.target

In [8]:
cancer.feature_names

array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error',
       'fractal dimension error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points',
       'worst symmetry', 'worst fractal dimension'], dtype='<U23')

### VotingClassifier로 개별모델은 로지스틱 회귀와 KNN을 보팅방식으로 결합하고 성능 비교
#### soft 방식

In [9]:
# from sklearn.ensemble import VotingClassifier

# 모델 2개 생성
model_lr = LogisticRegression() 
model_knn = KNeighborsClassifier(n_neighbors=8)

# voting 형식으로 모델 생성
# 모델명 = VotingClassifier( estimators = [('측정기변수', 모델생성자)   ])

model_vo = VotingClassifier( estimators=[('LR',model_lr),('KNN',model_knn)] , voting='soft' )
# model_vo = VotingClassifier( estimators=[('LR',LogisticRegression() ),('KNN',KNeighborsClassifier(n_neighbors=8))] , voting='soft' )


# 훈련 검증 분리
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, 
                                                    test_size=0.2 , random_state= 156)
# 학습
model_vo.fit(X_train , y_train)
model_vo.score(X_train , y_train)
model_vo.score(X_test , y_test)



C:\Users\TFG5076XG\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9473684210526315

In [10]:
# 앙상블과 상관없음.  


# 각 모델을 리스트로 생성해서 반복해서 학습 및 평가 데이터 확인
classifiers = [model_lr, model_knn]
for clf in classifiers:
    clf .fit(X_train , y_train)
    print(clf, clf.score(X_test , y_test))
    print(classification_report(y_test, clf.predict(X_test)))

LogisticRegression() 0.9385964912280702
              precision    recall  f1-score   support

           0       0.94      0.86      0.90        37
           1       0.94      0.97      0.96        77

    accuracy                           0.94       114
   macro avg       0.94      0.92      0.93       114
weighted avg       0.94      0.94      0.94       114

KNeighborsClassifier(n_neighbors=8) 0.9385964912280702
              precision    recall  f1-score   support

           0       0.89      0.92      0.91        37
           1       0.96      0.95      0.95        77

    accuracy                           0.94       114
   macro avg       0.93      0.93      0.93       114
weighted avg       0.94      0.94      0.94       114



C:\Users\TFG5076XG\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


#### hard 방식

In [11]:
# from sklearn.ensemble import VotingClassifier

model_lr = LogisticRegression() 
model_knn = KNeighborsClassifier(n_neighbors=8)

model_vo2 = VotingClassifier( estimators=[('LR',model_lr),('KNN',model_knn)] , voting='hard' )


X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, 
                                                    test_size=0.2 , random_state= 156)

model_vo2.fit(X_train , y_train)
model_vo2.score(X_train , y_train)
model_vo2.score(X_test , y_test)

C:\Users\TFG5076XG\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9385964912280702

In [12]:
# 앙상블과 상관없음.  


# 각 모델을 리스트로 생성해서 반복해서 학습 및 평가 데이터 확인
classifiers = [model_lr, model_knn]
for clf in classifiers:
    clf .fit(X_train , y_train)
    print(clf, clf.score(X_test , y_test))
    print(classification_report(y_test, clf.predict(X_test)))

LogisticRegression() 0.9385964912280702
              precision    recall  f1-score   support

           0       0.94      0.86      0.90        37
           1       0.94      0.97      0.96        77

    accuracy                           0.94       114
   macro avg       0.94      0.92      0.93       114
weighted avg       0.94      0.94      0.94       114

KNeighborsClassifier(n_neighbors=8) 0.9385964912280702
              precision    recall  f1-score   support

           0       0.89      0.92      0.91        37
           1       0.96      0.95      0.95        77

    accuracy                           0.94       114
   macro avg       0.93      0.93      0.93       114
weighted avg       0.94      0.94      0.94       114



C:\Users\TFG5076XG\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## voting = hard 에서 proba 사용 불가

In [13]:
# model_vo2.predict_proba(X_test[0])  => 오류

# Voting Regressor     => 회귀
- boston 데이터


In [14]:
from sklearn.datasets import load_boston

boston = load_boston()

boston_df = pd.DataFrame(boston.data , columns = boston.feature_names)
 
boston_df['PRICE'] = boston.target # y 값

C:\Users\TFG5076XG\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing datas

In [15]:
y_target = boston_df['PRICE']
X_data = boston_df.drop(['PRICE'], axis=1, inplace=False)

X_train , X_test , y_train , y_test = train_test_split( X_data , y_target , \
                                                       test_size=0.3, random_state=156)

In [16]:
# from sklearn.ensemble import VotingRegressor

# Voting  => sotf or hard



model_linear = LinearRegression()
model_lasso = Lasso(alpha=10)   # 알파값 10 줘서 낮아짐.
model_ridge = Ridge(alpha=10)

model_vo_r = VotingRegressor( estimators=[('LINEAR', model_linear),('LASSO', model_lasso), ('RIDGE', model_ridge)] )
# voting = 'soft' or 'hard'   => 오류


model_vo_r.fit(X_train , y_train)
print(model_vo_r.score(X_train , y_train) , model_vo_r.score(X_test , y_test))



0.6995242609530394 0.7382950882399059


In [17]:
# 사용한 알고리즘 
model_vo_r.estimators_

[LinearRegression(), Lasso(alpha=10), Ridge(alpha=10)]

In [18]:
# VotingRegressor의 파라미터 
model_vo_r.named_estimators_

{'LINEAR': LinearRegression(),
 'LASSO': Lasso(alpha=10),
 'RIDGE': Ridge(alpha=10)}

In [20]:
model_vo_r.score(X_test , y_test), r2_score(y_test, model_vo_r.predict(X_test))

(0.7382950882399059, 0.7382950882399059)

In [21]:
# 샘플데이터 예측가격
model_vo_r.predict([X_test.loc[15]])

C:\Users\TFG5076XG\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\TFG5076XG\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but Lasso was fitted with feature names
  warnings.warn(
C:\Users\TFG5076XG\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(


array([22.21449831])

In [ ]:
|